![alt text](Python07-iterators_extra/logo.png) 

# Итераторы, генераторы, декораторы

Тема непростая, будем слегка жертвовать формализмом и точностью в пользу понятности, а существование некотрых вещей вообще проигнорируем. Те из вас, кто захотят больше деталей и чётких формулировок найдут их в избытке в документации и интернете.

# Итераторы

Давайте вспомним цикл for, который мы прошли в лекции основы питона 2, и еще раз на него посмотрим.

In [ ]:
for number in range(5):
    print (number)

In [ ]:
for letter in 'Hello world':
    print (letter)

In [ ]:
for key in {'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           }:
    print (key)

In [ ]:
for line in open('Python07-iterators_extra/motivation.txt', 'r'):
    print (line)

In [ ]:
for element in [0,1,2,3,4,5]:
    print (element)

То есть мы что, можем идти циклом по любому объекту? А это вообще законно?

Давайте разбираться, как мы помним из лекции про ООП - все объекты являются классами. Рассмотрим их методы

In [ ]:
dir(range(5))

In [ ]:
dir({'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           })

Метод  *\_\_iter*\_\_  возвращает итератор для заданного объекта. ![alt text](Python07-iterators_extra/iterator.jpg) 

Итератор - это такой объект, который имплементирует протокол итерирования. Если сказать попроще, то итератор помнит текущее состояние и знает как выбирать следующий элемент объекта, над которым он построен.

In [ ]:
my_list = [0,1,2,3,4,5,6,7,8,9]
my_iter = iter(my_list)
print(my_iter)

Метод *\_\_next*\_\_ возвращает объект соответствующий текущему состоянию итератора и переходит к следующему, согласно заданному алгоритму (в тривиальном случае - k+1й объект). Делать это можно до тех пор, пока не получим ислючение **StopIteration**. С исключениями разберемся подробнее в лекции про Python Best Practices.

In [ ]:
print(my_iter.__next__()) # Выглядит жутковато

In [ ]:
print(next(my_iter)) # Вроде как-то попривычнее

In [ ]:
# Будем делать так, чтобы уменьшить синтаксическую нагрузку
next(my_iter) 

**Важное свойство итератора - он не хранит объект в памяти.**

In [ ]:
my_list = [0,1,2,3,4,5,6,7,8,9]
my_iter = iter(my_list)
print(next(my_iter))
print(next(my_iter))

In [ ]:
# Подменим объект не трогая итератор
my_list[3] = 'let put a string here'
my_list[4] = 'iterator does not care'

In [ ]:
# Выполним несколько раз
print(next(my_iter))

Метод  *\_\_iter*\_\_ может возвращать self (т.е. себя), то есть итератор от итератора - тоже итератор, причём тот же самый. ![alt text](Python07-iterators_extra/more_iterators.jpg) Мы в текущей лекции не будем уделять этому много внимания, но это очень важное и постоянно используемое свойство. Давайте взглянём:

In [ ]:
my_list = [0,1,2,3,4,5]
my_iter1 = iter(my_list)
my_iter2 = iter(my_iter1)

In [ ]:
print (my_iter1)
print (my_iter2)
print (my_iter1 == my_iter2)

In [ ]:
next(my_iter2)

In [ ]:
# Чтобы было понятнее, итераторы не просто одинаковые, это тот самый итератор!
next(my_iter1)

Если быть достаточно наблюдательным, то можно заметить, что метод *\_\_next*\_\_ вообще-то отсутствует у наших исходных объектов. 

Однако он появляется когда мы делаем из него итератор

In [ ]:
dir(my_list)

In [ ]:
dir(iter(my_list))

In [ ]:
# Как проверить объект на итерируемость
my_object = [1,2,3]
#my_object = 2
#my_object = 'hello'
try:
    iter(my_object)
    print('iterable')
except:
    print('not iterable')    

С try/except тоже будем разбираться в другой лекции. Если кратко и неправильно, то при ошибке вместо выпадания мы перескочим к исключению

Логика подсказывала, что итерироваться по числу как-то некорректно. Так и есть, делать этого нельзя.

Заметим, что итерироваться по объекту можно не только при помощи iter и next, но ещё и иначе, посредством метода *\_\_getitem*\_\_. Мы не будем его подробно рассматривать, отметим только что в отличии от iter который хранит состояние внутри итератора, getitem его не хранит, принимая его снаружи, со всеми вытекающими достоинствами и недостатками. В стародавние времена это был основной протокол итерирования (в python<2.4), теперь, видимо, можно считать его устаревшим и оставшимся нам по наследству во всех старых кодах. Просто не удивляйтесь при встрече.

In [ ]:
a = [12,13,14,15,16,17]
b = a.__getitem__(3)
print(b)

Давайте рассмотрим **range**, это даст нам новый взгляд на итераторы. Допустим, нам надо проитерироваться по какому-то очень большому циклу. Втупую, для этого пришлось бы записать всю последовательность элементов в память, создать итератор, и идти от предыдущего к следующему много раз. Но держать в памяти этот огромный объект не слишком-то хотелось бы. Тут нас и выручают итераторы, которые не держат в памяти объект, а знают только своё текущее состояние и закон по которому выбираем следующий элемент.

In [ ]:
# range не существует в каком-то полном запомненном/записанном виде
print(range(5)) 
print(type(range(5)))

In [ ]:
a = range(5)
# Но range на самом деле не итератор, это итерируемый объект
next(a)

In [ ]:
# А вот это - итератор по range
my_iter = iter(a) 

In [ ]:
next(my_iter)

In [ ]:
# Давайте проверим, а вдруг теперь когда мы сделали над range итератор, там всё же записали весь объект
my_iter

**Внимание, важно!**

Мы создали объект не записывая его в память и идём по нему, не держа в памяти ничего кроме текущего положения. Это очень быстро

In [ ]:
# Однако, если очень хочется, мы можем попросить его сделать. 
list(my_iter)

Все заметили что список начался не с нуля? Дело в том, что когда мы просим сделать, например, список от итератора, то происходит вот что, начиная с текущего состояния он начинает просить у итератора следующий элемент и писать в список, пока не дойдёт до конца. 

Ну, раз уж мы знаем ООП, давайте напишем свой класс с итератором. Это поможет нам лучше понять на что надо обращать внимание и как вообще их писать 

In [ ]:
class NaiveRange():
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.ind >= self.end:
            raise StopIteration  # С этим будем разбираться в другой лекции. Если просто - "упади с ТАКОЙ ошибкой"
        self.ind += 1
        return (self.ind - 1)

In [ ]:
my_range = NaiveRange(3,9)

In [ ]:
# Давайте ещё раз попросим список и рассмотрим детальнее.
list(my_range)

In [ ]:
# А что если ещё разок?
list(my_range)

Дело в том, что архитектура нашего класса не позволяет индексу вернуться к исходному состоянию без ре-инициализации, итератор пройден до конца, и с этим ничего не поделать. Такая итерация называется **исчерпаемой**. Пример: итерация по файлу. Мы можем его открыть и по мере прочтения достигнем конца. Конечно есть всякие трюки, но базово мы можем прочитать файл только раз, далее должны открыть его заново (и ре-инициализировать итератор).

Давайте теперь построим неисчераемый итератор.

In [ ]:
class BaseIter():
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __next__(self):
        if self.ind == self.end:
            raise StopIteration
        self.ind += 1
        return (self.ind -1)
    
    
class AdvancedRange():
    def __init__(self, start, end):
        self.end = end
        self.start = start
        
    def __iter__(self):
        return BaseIter(self.start, self.end)

In [ ]:
my_range = AdvancedRange(3,9)

In [ ]:
list(my_range)

In [ ]:
list(my_range)

Теперь, при вызове итератора, мы создаем объект класса BaseIter и можем так делать сколько угодно раз, и каждый из них будет держать в своей памяти своё состояние, не мешая остальным итерироваться. Это, соответственно, **неисчерпаемая** итерация. Пример: список, словарь.

Теперь, с учётом полученных сведений, давайте вернёмся к тому с чего начинали лекцию. 

**Псевдокод для цикла for** (он примерно так и организован)

```python
some_iter = iter(some_object)
while True:    
    try:        
        some_value = next(some_iter)        
    except StopIteration:        
        break        
    do_smth(some_value)
```

# Генераторы

Идея следующая. Сделаем такую функцию, которая будет по какому-то входному параметру создавать итераторы. Ключевой является команда **yield**, которая работает примерно как *return* только при следующем обращении проход по телу будет осуществляться не с начала, а с места где мы закончили. Разберём пару примеров и станет ясно зачем это всё надо. 
![alt text](Python07-iterators_extra/iter_vs_gen.svg)

In [ ]:
def my_first_genertor(var):
    yield 'first launch'
    yield 'second launch'
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [ ]:
my_gen = my_first_genertor(2)
#list(my_gen)

In [ ]:
next(my_gen)

Далее, сколько бы мы не вызывали, если не ре-инициализировать генератор, то он больше ничего не выдаст. Генератор **исчерпался**. 

**Return** же в свою очередь в генераторе не работает как обычно, а работает исключительно как *StopIteration*. Если return не написать, то он подразумевается в конце.

In [ ]:
def my_second_genertor(var):
    yield 'first launch'
    return 'second launch' # Заменим yield на return
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [ ]:
my_gen = my_second_genertor(2)
#list(gen)

In [ ]:
next(my_gen)

Заметим, что запустив повторно мы больше не переходим к следующему yield. Генератор исчерпался по исключению, не зависимо от того есть ли там ещё yield. 

**Важное замечание** Кроме описанного обычного завершения, стандартные генераторы можно завершить снаружи при помощи метода close. Этот метод вызывается, в том числе, принудительной остановкой и работает через исключение *GeneratorExit*. При самостоятельной реализации метода close, надо учитывать, что **GeneratorExit не отлавливается как базовое исключение**, и если не прописать его в явном виде при имплементации своего генератора, то есть шанс утратить возможность принудительной остановки, и никакие *Ctrl+C* не отработают. Уже после принудительной остановки, дальнейшие вызовы также получат *StopIteration*.

Выглядеть должно как-то так. Выглядит усложненным, но это близко к настоящей реализации.
```python
def safe_gen():
    closed = False
    try:
        yield 'check '
    except GeneratorExit:
        print('exit!')
        closed = True
        raise StopIteration
    finally:
        if not closed:
            print('worked!')

```

Зачем же нам всё это может нам понадобиться? Давайте вспомним одну классную *штуку* которую мы видели на второй лекции

In [ ]:
nums = ['zero', 'one', 'two', 'three', 'four']
for index, number in enumerate(nums):
    print(index, number)

**Enumerate** при итерации по объекту присваивает порядковые номера объектам, очень удобно и рекомендуется к использованию. *Enumerate* ни что иное как генератор, и может быть записано следующим образом

In [ ]:
def my_enumerate(iterable_object):
    ind = 0
    for smth in iterable_object:
        yield ind, smth
        ind += 1

In [ ]:
for index, number in my_enumerate(nums):
    print(index, number)

Когда встречаешь что-то такое первый раз, то невольно возникает мысль "Ну написали цикл в функции, и зачем это всё? Я и сам могу цикл написать и безо всяких генераторов". Однако, стоит обратить внимание на то, что генератор (он ведь тоже итератор!) не держит в памяти полностью записанный объект, вместо этого просто обращается к методу *next*, и это очень важно.

In [ ]:
print(my_enumerate(nums))

Давайте рассмотрим ещё несколько генераторов

In [ ]:
# map(func, smth_iterable) применяет к каждом элементу функцию func
double = map(lambda x: x + x, nums) 
print(double)

In [ ]:
print(list(double))

In [ ]:
# Потренируемся, сделаем свой map
def my_map(func, iterable_object):
    for smth in iterable_object:
        yield func(smth)

In [ ]:
my_double = my_map(lambda x: x + x, nums) 
print(list(my_double))

In [ ]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
len_filter = filter(lambda x: len(x) > 3, nums)
print(list(len_filter))

In [ ]:
# Cделаем свой filter
def my_filter(condition, iterable_object):
    for smth in iterable_object:
        if condition(smth):
            yield smth

In [ ]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
my_len_filter = my_filter(lambda x: len(x) > 3, nums)
print(list(my_len_filter))

Есть множество полезных генераторов, которые пока ещё не впилили в основной питон (но судя по логам версий - этим занимаются): count, repeat, cycle, chain и другие. Их можно найти в пакете itertools. Давайте посмотрим.

In [ ]:
from itertools import repeat
# Повторяет то что ему дали

In [ ]:
for var in repeat(5):
    print(var)

In [ ]:
from itertools import chain
# "Раскрывает" итерируемые объекты и делает из них один. Но делает это только на 1 уровень в "глубину"

In [ ]:
list(chain('A', 'B', [1, 2, 3, None], 'a string', range(4,7)))

In [ ]:
list(chain('A', 'B', [1, 2, 3, 'a string', None], [range(4,7)]))

In [ ]:
from itertools import count
# Принимает начальное состояние и шаг и делает с ними бесконечный цикл

In [ ]:
for i in count(0,3):
    print(i)

Давайте еще рассмотрим такой пример. Допустим у нас есть объект, и мы хотим его перегруппировать в процессе итерации. 

In [ ]:
my_object = [0, 1, 2, 'zero', 'one', 'two', 'A', 'B', 'C']
my_iter = iter(my_object)
print(list(zip(my_iter, my_iter, my_iter)))

### Выражения-генераторы

Стандарт Pep-289 даёт нам подробные инструкции как надо записывать и вообще как пользоваться выражениями-генераторами https://www.python.org/dev/peps/pep-0289/ 
Например,

In [ ]:
def gen(exp):
    for x in exp:
        yield x**2
g = gen(iter(range(10)))

# Эквивалентно
g = (x**2 for x in range(10))

Это удобно краткой записью в одну строку, при этом это не просто выражение, а генератор, со всеми вытекающими преимуществами и свойствами, которые мы ранее изучили. 

**Расширим функциональность генератора**. Теперь мы хотим не только получать из него следующий элемент по команде, но и как-то более явно в этом участвовать, например посылать в него данные, чтобы получать другие данные. Для этого Используем команду **send**, а принимать данные будем при помощи всё того же *yield*, но немного иначе.

In [ ]:
def gen():
    for i in range(10):
        x = yield 
        print(i + x)

In [ ]:
my_gen = gen()
next(my_gen) 

Могло показаться что мы попросили следующий элемент... но это не так, это обращение к генератору для его инициализации. На самом деле next теперь вообще не работает, ввиду более сложной конструкции.

In [ ]:
next(my_gen)

NoneType - это потому что x - None, он не пришёл, и с ним невозможно сложиться.

Послать x можно при помощи send

In [ ]:
my_gen.send(17)

next(my_gen) эквивалентент my_gen.send(None)

![alt text](Python07-iterators_extra/deeper.png) 

Давайте теперь рассмотрим такой пример, когда у нас есть два генератора, которые общаются между собой 

In [ ]:
import random

# Генерим новые данные... но это не генератор! (по определению генератора)
def get_data():
    return random.sample(range(10), 2)

# Обрабатываем данные
def consume():
    total_sum = 0
    total_amount = 0
    
    while True:
        data = yield
        total_sum += sum(data)
        total_amount += len(data)
        print('Running average is {}'.format(total_sum/total_amount))
        
# Производим новые данные и передаём в обработку
def produce(consumer):
    while True:
        data = get_data()
        print('New data {}'.format(data))
        consumer.send(data)
        yield

In [ ]:
consumer = consume()
next(consumer)
producer = produce(consumer)

for _ in range(1000):
    next(producer)

Чем хороша приведённая конструкция (её идея)? Мы не просто разделили процесс генерации и обработки данных, это можно было сделать и без всяких генераторов. **Send** создаёт отдельный поток вычислений, которые производятся параллельно основному телу программы. Таким образом, генерить и обрабатывать данные многими разными способами можно в параллели, притормаживая только по ключевому слову *yield* чтобы дождаться данных или следующего обращения. 

Всё это называется Coroutine и поподробнее можно почитать в PEP-342 https://www.python.org/dev/peps/pep-0342/

![alt text](Python07-iterators_extra/coroutine.jpg)

Картинка выглядит похожей, и может показаться что мы переименовали прерывания и переключения на yield, и огород не стоил свеч. Но, повторимся, send уходит в отдельный поток вычислений.

Эта идея позволяет нам, например, создать планировщик задач, который по определенным условиям будет запускать тот или иной генератор, и экономить время выполняя их в параллели и переключаясь на них только когда это требуется. Это наша первая встреча с мультипроцессингом. На самом деле тут очень много нюансов, и обсуждать это мы будем в самой последней лекции. 

**ВНИМАНИЕ** *Те кто переживает что плохо понял тему, и кто хочет разобраться и закрепить в памяти материал по генераторам, могут после лекции попробовать написать генератор, который в параллели качнет наши прошедшие лекции с гитхаба, обложить таймерами, и убедиться что это действительно работает быстрее, а так же убедиться что это почему-то быстрее не в N раз, где N - число прошедших лекций. Это не сложно, и это не обязательное задание, но это поможет усвоить материал. Ничего никуда посылать не надо.*

Последнее о генераторах, что хотелось бы рассмотреть это конструкция **yield from**. Обычно говорят, что эта конструкция устанавливает двунаправленную связь между вызывающим генератором и суб-генератором, передавая, в том числе, исключения (transparent two way channel).

Сначала простой пример:

In [ ]:
def generator_simple():
    for i in range(10):
        yield i
    for j in range(10, 20):
        yield j

Он, будет выдавать нам числа от 0 до 19. И мы решили разбить его на две части чтобы их можно было еще использовать по отдельности где-то в других местах и не писать по генератору на все случаи жизни.

In [ ]:
def generator1():
    for i in range(10):
        yield i

def generator2():
    for j in range(10, 20):
        yield j        

А потом всё же решили что нам нужен один генератор, который умеет от 0 до 19

In [ ]:
def generator():
    for i in generator1():
        yield i
    for j in generator2():
        yield j

In [ ]:
# То же самое можно записать в виде
def generator_from():
    yield from generator1()
    yield from generator2()

Понятно, что это не очень впечатляет, так что давайте сразу посмотрим чуть более сложный пример

In [ ]:
def writer():
    """A coroutine that writes data *sent* to it """
    while True:
        w = (yield)
        print('>> ', w)

def writer_wrapper(coro):
    coro.send(None)  # Для инициализации
    while True:
        try:
            x = (yield)  # Принимаем
            coro.send(x)  # и передаём
        except StopIteration:
            pass

In [ ]:
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

In [ ]:
#Всё то же самое выполняется при помощи yield from
def writer_wrapper_from(coro):
    yield from coro

In [ ]:
w = writer()
wrap = writer_wrapper_from(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

Если подытожить, то yield from позволяет серьезно упростить код и не думать о всяких вещах, о которых мы обычно не хотим думать, и **не теряет** при этом никаких важных исключений или типа того.

# Декораторы
![alt text](Python07-iterators_extra/decorators.jpg)

Допустим, мы столкнулись с задачей, что нам надо подправить кое-какие функции, но лезть в них и менять код прямо там нам не сильно охота, потому что, например, мы можем потом передумать. Или переживаем за общую совместимость кода в большом проекте. Сделаем обёртку нужной нам функции:

In [ ]:
def deprecated(func):
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

In [ ]:
def my_func(x):
    return x+3

# "Обернём" нашу функцию посредством переприсвоения
my_func = deprecated(my_func)

In [ ]:
my_func(2)

In [ ]:
# Ровно то же самое можно записать без переприсвоения:
@deprecated # Это декоратор
def my_func(x):
    return x+3

In [ ]:
my_func(2)

In [ ]:
# Но, заметим, что именем ф-ии стало имя обёртки
my_func.__name__

Чтобы сделать красивый декоратор и совсем ничего в функции не менять, кроме того что мы поменять хотим, давайте воспользуемся библиотекой functools ![alt text](Python07-iterators_extra/decoration.jpeg)

In [ ]:
import functools

def deprecated(func):
    @functools.wraps(func)
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

@deprecated
def my_func(x):
    return x+3

my_func.__name__

Functools - высокоуровневая библиотека, упрощающая работу с функциями и другими вызываемыми объектами, когда требуется расширить, дополнить или даже полностью изменить сам объект. Более детально можно познакомиться, например, здесь https://docs.python.org/3.3/library/functools.html

Декораторы работают с классами так же как и с функциями. 

Создадим декоратор выводящий время работы декорируемой функции или класса, вполне жизненный пример. 

In [ ]:
import time
import functools

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer


In [ ]:
class TimeWaster:    
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])        

In [ ]:
tw = TimeWaster(1000)
tw.waste_time(999)

Допустим, мы хотим чтобы наш декоратор что-то нам важное вывел, например исключение. Только не в общий вывод, чтобы не засорять, а куда-то в другое место, где мы при необходимости прочитаем. Для этого можно использовать *атрибуты декоратора*

In [ ]:
def redirected_output(destination=None):        
    def wrapper(*args, **kwargs):
        with open(destination, 'w') as output:
            print('Some important information', 
                  file=output)    
    return wrapper

In [ ]:
@redirected_output(destination='log.txt')
def foo():
    pass

Этот пример не очень хорош, зато он прост и демонстрирует, что в сам декоратор можно передавать входные данные

![alt text](Python07-iterators_extra/end.jpg)

# Домашнее задание.

Вы снова получите у телеграм-бота файлы с автомобильными траекториями. По полученным данным нужно выявить нарушителей скоростного режима. Предельная допустимая скорость устанавливается в 60 км/ч. Сделать это нужно при помощи полученных сегодня знаний **(pandas запрещен)**, написать хороший итератор или несколько, и пройтись ним по данным.

Генератор принимает путь к файлу и выдаёт все объекты превышающие допустимую скорость 60 км/ч непрерывно в течении 1 секунды или дольше.

Скорость это путь разделенный на время.  

**Координаты в метрах, время в секундах.**

Для самопроверки нужно передать боту ID всех нарушителей. Для этого нужно записать их в файл по одному в строку и прислать.

Детали:
Мы ожидаем от вас answer.txt с ответами и code.py с кодом **(не ноутбук, не переименованный ноутбук, а именно текстовый файл)**, в котором будет написан итератор/генератор, и всё что нужно для запуска.

Давайте посмотрим пример (это не решение, это фейковый пример на фейковых данных!)

In [ ]:
import sys
import os 
sys.path.append('./Python07-iterators_extra')
from hometask_format_example import my_generator

In [ ]:
list(my_generator('datafile.csv'))